# Capstone Project: Grab Challenge(Traffic Management)

In [1]:
import pandas as pd
from datetime import datetime,timedelta

import geohash as gh
from math import sin, cos, radians, atan2,sqrt
import statsmodels.api as sm
import numpy as np

#Visualization
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
import seaborn as sns
from mpl_toolkits.basemap import Basemap

import progressbar
from time import sleep

# 8. Using Facebook Prophet

In [9]:
!pip install Cython

In [10]:
!pip install pystan

In [ ]:
import pystan
model_code = 'parameters {real y;} model {y ~ normal(0,1);}'
model = pystan.StanModel(model_code=model_code)  # this will take a minute
y = model.sampling(n_jobs=1).extract()['y']
y.mean()  # should be close to 0

In [ ]:
#!pip install --upgrade setuptools

In [6]:
!pip install fbprophet

  Using cached https://files.pythonhosted.org/packages/33/fb/ad98d46773929079657706e6b2b6e366ba6c282bc2397d8f9b0ea8e5614c/fbprophet-0.5.tar.gz
  Using cached https://files.pythonhosted.org/packages/74/83/d0fa07078f4d4ae473a89d7d521aafc66d82641ea0af0ef04a47052e8f17/convertdate-2.1.3-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/05/97/dd99fa9c0d9627a7b3c103a00f1566d8193aca8d473884ed258cca82b06f/setuptools_git-1.2-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/c3/2c/9e1a815add6c222a0d4bf7c644e095471a934a39bc90c201f9550a8f7f14/ephem-3.7.6.0.tar.gz
  Complete output from command C:\Users\Acer\Anaconda\python.exe -u -c "import setuptools, tokenize;__file__='C:\\Users\\Acer\\AppData\\Local\\Temp\\pip-install-x6tojd76\\fbprophet\\setup.py';f=getattr(tokenize, 'open', open)(__file__);code=f.read().replace('\r\n', '\n');f.close();exec(compile(code, __file__, 'exec'))" bdist_wheel -d C:\Users\Acer\AppData\Local\Temp\pip-wheel-nws8jndj --p

  Failed building wheel for fbprophet
  Failed building wheel for ephem
Command "C:\Users\Acer\Anaconda\python.exe -u -c "import setuptools, tokenize;__file__='C:\\Users\\Acer\\AppData\\Local\\Temp\\pip-install-x6tojd76\\ephem\\setup.py';f=getattr(tokenize, 'open', open)(__file__);code=f.read().replace('\r\n', '\n');f.close();exec(compile(code, __file__, 'exec'))" install --record C:\Users\Acer\AppData\Local\Temp\pip-record-lxayltq6\install-record.txt --single-version-externally-managed --compile" failed with error code 1 in C:\Users\Acer\AppData\Local\Temp\pip-install-x6tojd76\ephem\


In [37]:
from fbprophet import Prophet



ModuleNotFoundError: No module named 'fbprophet'

In [20]:
df_prophet = pd.read_csv('Averaged_All_locs.csv')

df_prophet.Timestamp = df_prophet.Timestamp.apply(lambda x: pd.Timestamp(x))

df_prophet.rename(columns={'Timestamp':'ds','demand':'y'}, inplace = True)

df_prophet.sort_values('ds',inplace=True)

## Train Test Split

In [32]:
df_train = df_prophet.iloc[:4695]
df_test = df_prophet.iloc[4695:]

In [35]:
df_train.head()

,ds,y
0,2019-01-01 00:00:00,0.022744
1,2019-01-01 00:15:00,0.025120
2,2019-01-01 00:30:00,0.027232
3,2019-01-01 00:45:00,0.029258
4,2019-01-01 01:00:00,0.033303


In [36]:
df_test.head()

,ds,y
4695,2019-02-19 00:00:00,0.066286
4696,2019-02-19 00:15:00,0.069983
4697,2019-02-19 00:30:00,0.072283
4698,2019-02-19 00:45:00,0.072972
4699,2019-02-19 01:00:00,0.074352


## Create and Fit Model

In [ ]:
m = Prophet(yearly_seasonality=False)
m.fit(df_prophet);

In [ ]:
future = m.make_future_dataframe(periods=720, freq='H')
forecast = m.predict(future)

In [ ]:
m.plot(forecast, xlabel='Date', ylabel='Number of Uber Pickups')
plt.title('Prophet - Model fit and 1-month prediction')

In [ ]:
m.plot_components(forecast)

In [ ]:
pred = pd.DataFrame(forecast.yhat[-720:])
plot_forecast(ts15_test.pickups, pred)
rmse(ts15_test.pickups, pred)